# Caderno 1 - Tratamento das bases de dados

A base [juris_tcu](https://github.com/marcusborela/ind-ir/tree/main/data/juris_tcu) será usada como padrão ouro para verificar evoluções na pesquisa de jurisprudência selecionada. Ela contém três arquivos:

- doc.csv: CSV contendo todos os documentos da jurisprudência selecionada (a data da base é, aproximadamente, junho de 2023)
- query.csv: Lista de queries de referência
- qrel.csv: Lista de documentos relevantes por query

A base [jurisprudência selecionada](https://sites.tcu.gov.br/jurisprudencia/) contém a lista de todos os documentos da jurisprudência selecionada. O download foi feito dia 8/4. Devido à dificuldade de se guardar arquivos grandes no github, dividi o arquivo original em quatro partes.

Em tese, não temos que trabalhar com a base de jurisprudência selecionada, e sim apenas com a base juris_tcu. Entretanto, embora todos os documentos da base juris_tcu estejam na base jurisprudencia-selecionada, o formato está um pouco diferente:

- A base jurisprudencia-selecionada contém mais campos
- A KEY na base jurisprudencia-selecionada está no formato final da base (e a do juris-tcu está em um formato intermediário)

Assim, o que será feito neste caderno é o tratamento da base juris_tcu para:

- Considerar a KEY dos documentos no formato final (e não no formato intermediário)
- Considerar o documento completo em vez de um documento parcial

## 1. Carrega as bases de dados

In [1]:
import pandas as pd

PASTA_DADOS = './dados/'
PASTA_JURIS_TCU = f'{PASTA_DADOS}originais/juris_tcu/'
PASTA_JS = f'{PASTA_DADOS}originais/jurisprudencia_selecionada/'
PASTA_EXPORTACAO_JURIS_TCU = f'{PASTA_DADOS}outputs/1_tratamento_juris_tcu/'

# Carrega os arquivos 
def carrega_juris_tcu():
    doc = pd.read_csv(f'{PASTA_JURIS_TCU}doc.csv')
    query = pd.read_csv(f'{PASTA_JURIS_TCU}query.csv')
    qrel = pd.read_csv(f'{PASTA_JURIS_TCU}qrel.csv')

    return doc, query, qrel

def carrega_jurisprudencia_selecionada():
    js_parte_1 = pd.read_csv(f'{PASTA_JS}jurisprudencia-selecionada-parte-1.csv', sep='|')
    js_parte_2 = pd.read_csv(f'{PASTA_JS}jurisprudencia-selecionada-parte-2.csv', sep='|')
    js_parte_3 = pd.read_csv(f'{PASTA_JS}jurisprudencia-selecionada-parte-3.csv', sep='|')
    js_parte_4 = pd.read_csv(f'{PASTA_JS}jurisprudencia-selecionada-parte-4.csv', sep='|')
    
    js_restaurado = pd.concat([js_parte_1, js_parte_2, js_parte_3, js_parte_4], ignore_index=True)

    return js_restaurado
    

In [2]:
doc, query, qrel = carrega_juris_tcu()
js = carrega_jurisprudencia_selecionada()

print('********** JURIS_TCU ********** ')
print(f'Total de documentos: {len(doc)}')
print(f'Total de query: {len(query)}')
print(f'Total de qrel: {len(qrel)}')

print('********** JURISPRUDENCIA_SELECIONADA ********** ')
print(f'Total de documentos: {len(js)}')

********** JURIS_TCU ********** 
Total de documentos: 16045
Total de query: 150
Total de qrel: 2250
********** JURISPRUDENCIA_SELECIONADA ********** 
Total de documentos: 16449


## 2. Relação entre as chaves dos documentos nas bases juris-tcu e jurisprudencia-selecionada

Inicialmente, é necessário separar corretamente os documentos da base jurisprudencia-selecionada de acordo com os documentos da base juris-tcu.

A KEY na jurisprudencia-selecionada é formada por um prefixo e um ID. O ID corresponde ao ID da base juris-tcu.

Para checar que esse raciocínio está correto, vamos primeiro separar a KEY em prefixo e número:

In [3]:
js[['KEY_PREFIXO', 'KEY_NUMERO']] = js.KEY.str.rsplit('-', n=1, expand=True)
# Converte KEY_NUMERO para inteiro:
js.KEY_NUMERO = js.KEY_NUMERO.astype(int)

Agora vamos verificar quantos tipos de prefixo existem (2) e verificar se todos os números são únicos:

In [4]:
print('PREFIXOS ÚNICOS:')
print(js.KEY_PREFIXO.unique())

print(f'Tamanho da base: {len(js)}')
print(f'Total de KEY_NUMERO únicos: {len(js.KEY_NUMERO.unique())}')

PREFIXOS ÚNICOS:
['JURISPRUDENCIA-SELECIONADA' 'JURISPRUDENCIA-SELECIONADA-LEGADA']
Tamanho da base: 16449
Total de KEY_NUMERO únicos: 16449


Antes de trabalhar com o merge das bases, vamos checar se todos os IDs na base juris-tcu (doc.ID) possui uma correspondência na base jurisprudência selecionada.

Para isso, basta checar:

- Se len(doc.ID) == len(doc.ID.unique()) => Todos os IDs são únicos
- Se o total de correspondências de js.KEY_NUMERO em doc.ID é igual ao tamanho de len(doc.ID)

In [5]:
correspondencias = js.KEY_NUMERO.isin(doc.ID)

print(f'len(doc.ID) = {len(doc.ID)}')
print(f'len(doc.ID.unique()) = {len(doc.ID.unique())}')
print(f'correspondencias = {correspondencias.sum()}')

len(doc.ID) = 16045
len(doc.ID.unique()) = 16045
correspondencias = 16045


Apenas para garantir, podemos verificar também checando os limites mínimo e máximo dos intervalos das IDs para cada PREFIXO (JURISPRUDENCIA-SELECIONADA e JURISPRUDENCIA_SELECIONADA-LEGADA):

In [6]:
filtro_js = ~js.KEY.str.contains('JURISPRUDENCIA-SELECIONADA-LEGADA')
filtro_js_legada = js.KEY.str.contains('JURISPRUDENCIA-SELECIONADA-LEGADA')

# Encontrar o menor valor de 'Numero' para STRING1 e STRING2
min_js = js[filtro_js].KEY_NUMERO.min()
min_js_legada = js[filtro_js_legada].KEY_NUMERO.min()
max_js = js[filtro_js].KEY_NUMERO.max()
max_js_legada = js[filtro_js_legada].KEY_NUMERO.max()

print(f'Intervalo de IDs para JURISPRUDENCIA_SELECIONADA: {min_js} a {max_js}')
print(f'Intervalo de IDs para JURISPRUDENCIA_SELECIONADA_LEGADA: {min_js_legada} a {max_js_legada}')

Intervalo de IDs para JURISPRUDENCIA_SELECIONADA: 284 a 165421
Intervalo de IDs para JURISPRUDENCIA_SELECIONADA_LEGADA: 5 a 250


## 3. Corrigindo as chaves dos documentos na base juris-tcu

A nova base doc terá todos os atributos que tem a base jurisprudencia-selecionada. Os registros serão os mesmo. Assim, filtra a base de jurisprudencia-selecionada para ter apenas os registros que tem na base juris-tcu:

In [7]:
print('********** ANTES DO PROCESSO **********')
print(f'Tamanho da base juris-tcu: {len(doc)}')
print(f'Colunas da base juris-tcu: {doc.columns}')

correspondencias = js.KEY_NUMERO.isin(doc.ID)
doc = js[correspondencias]

print('********** DEPOIS DO PROCESSO **********')
print(f'Tamanho da base juris-tcu: {len(doc)}')
print(f'Colunas da base juris-tcu: {doc.columns}')

********** ANTES DO PROCESSO **********
Tamanho da base juris-tcu: 16045
Colunas da base juris-tcu: Index(['ID', 'TEXT', 'REFERENCE_LIST', 'PARADIGMATIC', 'AREA_NAME',
       'AREA_ID_DESCRIPTOR', 'NORMATIVE_PROCESS_TYPE',
       'NORMATIVE_IDENTIFICATION', 'NORMATIVE_DATE', 'NORMATIVE_AUTHOR_TYPE',
       'NORMATIVE_AUTHOR_NAME'],
      dtype='object')
********** DEPOIS DO PROCESSO **********
Tamanho da base juris-tcu: 16045
Colunas da base juris-tcu: Index(['KEY', 'NUMACORDAO', 'ANOACORDAO', 'COLEGIADO', 'AREA', 'TEMA',
       'SUBTEMA', 'ENUNCIADO', 'EXCERTO', 'NUMSUMULA', 'DATASESSAOFORMATADA',
       'AUTORTESE', 'FUNCAOAUTORTESE', 'TIPOPROCESSO', 'TIPORECURSO',
       'INDEXACAO', 'INDEXADORESCONSOLIDADOS', 'PARAGRAFOLC',
       'REFERENCIALEGAL', 'PUBLICACAOAPRESENTACAO', 'PARADIGMATICO',
       'KEY_PREFIXO', 'KEY_NUMERO'],
      dtype='object')


Não há nada que precisa ser feito na base de query. Já para a base qrel, é necessário corrigir o DOC_ID, adicionando o prefixo correto.

Como os limites são bem definidos, podemos formar a KEY assim:

- Se DOC_ID for menor ou igual a 250, JURISPRUDENCIA-SELECIONADA-LEGADA-DOC_ID
- Se DOC_ID for maior que 250, JURISPRUDENCIA-SELECIONADA-DOC_ID

In [8]:
qrel['DOC_KEY'] = qrel['DOC_ID'].apply(lambda x: f"JURISPRUDENCIA-SELECIONADA-LEGADA-{x}" if x <= 250 else f"JURISPRUDENCIA-SELECIONADA-{x}")

print('Testando duas chaves em cada grupo:')
print(qrel[qrel.DOC_ID == 21064].DOC_KEY)
print(qrel[qrel.DOC_ID == 13].DOC_KEY)


Testando duas chaves em cada grupo:
0       JURISPRUDENCIA-SELECIONADA-21064
1066    JURISPRUDENCIA-SELECIONADA-21064
Name: DOC_KEY, dtype: object
221    JURISPRUDENCIA-SELECIONADA-LEGADA-13
Name: DOC_KEY, dtype: object


Agora, apenas por uma questão de normalização, vamos usar a expressão KEY em vez da expressão ID nas bases doc, query e qrel:

In [9]:
# Na base doc o nome já é KEY. Vamos apenas apagar KEY_PREFIXO e KEY_NUMERO:
doc = doc.drop(['KEY_PREFIXO', 'KEY_NUMERO'], axis=1, errors='ignore')

# Renomeia a coluna ID da query para KEY
query.rename(columns={'ID': 'KEY'}, inplace=True)

# Apaga a coluna DOC_ID na base qrel e renomeia QUERY_ID para QUERY_KEY
# e reordena para QUERY_KEY e DOC_KEY ficarem no início
qrel.rename(columns={'QUERY_ID': 'QUERY_KEY'}, inplace=True)
qrel = qrel.drop(['DOC_ID'], axis=1, errors='ignore')
qrel = qrel[['QUERY_KEY', 'DOC_KEY', 'SCORE', 'ENGINE', 'RANK']]
# Checa as colunas
print(doc.columns)
print(query.columns)
print(qrel.columns)

Index(['KEY', 'NUMACORDAO', 'ANOACORDAO', 'COLEGIADO', 'AREA', 'TEMA',
       'SUBTEMA', 'ENUNCIADO', 'EXCERTO', 'NUMSUMULA', 'DATASESSAOFORMATADA',
       'AUTORTESE', 'FUNCAOAUTORTESE', 'TIPOPROCESSO', 'TIPORECURSO',
       'INDEXACAO', 'INDEXADORESCONSOLIDADOS', 'PARAGRAFOLC',
       'REFERENCIALEGAL', 'PUBLICACAOAPRESENTACAO', 'PARADIGMATICO'],
      dtype='object')
Index(['KEY', 'TEXT', 'SOURCE'], dtype='object')
Index(['QUERY_KEY', 'DOC_KEY', 'SCORE', 'ENGINE', 'RANK'], dtype='object')


## 4. Exporta os arquivos do juris_tcu tratados

In [10]:
# Vamos exportar os doc em 4 partes para facilitar subir pro github:
doc[0:4000].to_csv(f'{PASTA_EXPORTACAO_JURIS_TCU}doc_tratado_parte_1.csv', sep='|', index=False)
doc[4000:8000].to_csv(f'{PASTA_EXPORTACAO_JURIS_TCU}doc_tratado_parte_2.csv', sep='|', index=False)
doc[8000:12000].to_csv(f'{PASTA_EXPORTACAO_JURIS_TCU}doc_tratado_parte_3.csv', sep='|', index=False)
doc[12000:].to_csv(f'{PASTA_EXPORTACAO_JURIS_TCU}doc_tratado_parte_4.csv', sep='|', index=False)
query.to_csv(f'{PASTA_EXPORTACAO_JURIS_TCU}query_tratado.csv', sep='|', index=False)
qrel.to_csv(f'{PASTA_EXPORTACAO_JURIS_TCU}qrel_tratado.csv', sep='|', index=False)